# Base types

A `Container` is an object that can contain other objects, such as a room, a box or a backpack. Objects (and people!) can be placed inside these. For now, we don't care about size - a person can fit inside a pocket.

In [159]:
class Container(object):
    def __init__(self):
        self.content = []
        
    def place_object_here(self, item):
        self.content.append(item)
        item.update_position(self)
        return self
        
    def remove(self, item):
        self.content.remove(item)
        return self

    def find(self, name):
        return [item for item in self.content if item.name == name]
        return self
        
    def __repr__(self):
        items = [str(item) for item in self.content if not isinstance(item, User)]
        if len(items) == 0:
            return "nothing"
        return "\n".join(items)
        

A `PlacableObject` is any object (or person) that can be placed inside a `Container`. It remembers where it currently is placed.

In [160]:
class PlacableObject(object):
    def __init__(self, name):
        self.name = name
        self.position = None
    
    def place(self, room):
        room.place_object_here(self)
        return self
    
    def update_position(self, room):
        if self.position is not None:
            self.position.remove(self)
        self.position = room
        return self
        
    def __repr__(self):
        return self.name

# Game object types
A `Room` is a container that is connected to other rooms using doors, making up a map. People can walk between rooms using the doors. Note that doors needn't be bidirectional - some rooms might have one way doors, trapping you.

In [161]:
class Room(Container):
    def __init__(self, description):
        Container.__init__(self)
        self.description = description
        self.doors = {}
        
    def make_door(self, other, direction):
        self.doors[direction] = other
        return self
    
    def __repr__(self):
        return """%s
There are doors in these directions: %s
In this room:
%s
""" % (self.description,
       ", ".join(self.doors.keys()),
       Container.__repr__(self))
    

A `Person` is both a `PlacableObject` in that it can reside in e.g. a `Room`, but it's also a `Container` so that it can hold stuff. `Person` has methods to pick up things from the room they are in given some name of the thing to pick up, and to drop things they are holding (placing them in the room the person is in).

In [ ]:
class Person(Container, PlacableObject):
    def __init__(self, name):
        Container.__init__(self)
        PlacableObject.__init__(self, name)
        
    def pick(self, name):
        items = self.position.find(name)
        assert len(items) > 0, "Item not found"
        for item in items:
            item.place(self)
        return self
    
    def drop(self, name):
        items = self.find(name)
        assert len(items) > 0, "Item not found"
        for item in items:
            item.place(self.position)
        return self

    def walk(self, direction):
        self.place(self.position.doors[direction])
        return self

    def __repr__(self):
        return """%s who has %s
""" % (self.name, Container.__repr__(self))

A `User` is a kind of `Person` who knows they are the center of their universe. This is mostly a printing issue, to make plaing the game look nice.

In [184]:
class User(Person):
    def __repr__(self):
        return """You are in
%s

You have: %s
""" % (self.position, Container.__repr__(self))

# Making a map and placing people and things in it

To make a map, we create the rooms and connect them using doors.

In [203]:
livingroom = Room("Livingroom")
bedroom = Room("Bedroom")
kitchen = Room("Kitchen")
bedroom.make_door(livingroom, "west"); livingroom.make_door(bedroom, "east");
kitchen.make_door(livingroom, "north"); livingroom.make_door(kitchen, "south");


Once we have the basic map made, we can create and place some objects and people in the map.

In [204]:
spoon = PlacableObject("spoon")
spoon.place(kitchen)

ove = Person("Ove")
ove.place(livingroom)

Ove who has nothing

# Creating a user and beginning the game

To start playing, we need to create a user object and place it somewhere on the map. After that, we can just use the user object to walk around and interact with the game.

In [205]:
me = User("Egil")
me.place(kitchen)

You are in
Kitchen
There are doors in these directions: north
In this room:
spoon


You have: nothing

In [206]:
me.pick("spoon")

You are in
Kitchen
There are doors in these directions: north
In this room:
nothing


You have: spoon

In [207]:
me.drop("spoon")

You are in
Kitchen
There are doors in these directions: north
In this room:
spoon


You have: nothing

In [208]:
me.pick("spoon")

You are in
Kitchen
There are doors in these directions: north
In this room:
nothing


You have: spoon

In [209]:
me.walk("north")

You are in
Livingroom
There are doors in these directions: east, south
In this room:
Ove who has nothing



You have: spoon

In [210]:
me.walk("south")

You are in
Kitchen
There are doors in these directions: north
In this room:
nothing


You have: spoon

In [211]:
me.walk("north")

You are in
Livingroom
There are doors in these directions: east, south
In this room:
Ove who has nothing



You have: spoon

In [212]:
me.walk("east")

You are in
Bedroom
There are doors in these directions: west
In this room:
nothing


You have: spoon

In [213]:
me.walk("east")

KeyError: 'east'

In [214]:
ove.walk("south")

Ove who has nothing

In [215]:
ove.position

Kitchen
There are doors in these directions: north
In this room:
Ove who has nothing
